# Sand pit

When I worked on AoC 2015, I noticed I can do better than that. I had no idea what to expect, and my approach was horrible. After reading the assignment, I had to search the internet regarding certain features in python. However, I did not want to just sit down and read all the python documentations from the start to the end. So I got some important highlight of the language to focus on. So, I decided to get some hands on training on the toolset that Peter Norvig <a href='https://nbviewer.org/github/norvig/pytudes/blob/main/ipynb/Advent-2021.ipynb'>used</a>. After, I will solve the puzzles, then compare mine with his and see how horrible mine is.

In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

## Playing with imported libraries

### Annotations

<code>annotations</code> delays evaluation of function annotations. Without the future import, NameError is raised below. However, after the import, it works without any issue.

In [2]:
class A:
    def a(self) -> A:
        return

### Collections

#### Counter 

Counter can be used to count things.

There are many ways to initiate the Counter.

In [2]:
Counter()

Counter()

In [3]:
Counter('Darkness')

Counter({'D': 1, 'a': 1, 'r': 1, 'k': 1, 'n': 1, 'e': 1, 's': 2})

In [4]:
Counter({'bacon': 4, 'eggs': 30})

Counter({'bacon': 4, 'eggs': 30})

In [5]:
c = Counter(onions=2, cucumber=19, potato=11, sweet_potato=38)
c

Counter({'onions': 2, 'cucumber': 19, 'potato': 11, 'sweet_potato': 38})

Counts can be retrieved by using keys. If it does not exist in the Counter, count is 0.

In [6]:
c['onions'], c['zombie']

(2, 0)

Counts can be changed. It can be 0 or negative. 

In [7]:
c['sweet_potato'] = 0
'sweet_potato' in c

True

In [8]:
c['sweet_potato'] = -100
'sweet_potato' in c

True

Elements in the Counts can be deleted with del. 

In [9]:
del c['sweet_potato']
'sweet_potato' in c

False

Counter has help methods. <code>elements</code> returns an iterator that generates all the elements.

In [10]:
list(c.elements())

['onions',
 'onions',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato']

Most common items from top to bottom.

In [11]:
c.most_common()

[('cucumber', 19), ('potato', 11), ('onions', 2)]

In [12]:
c.most_common(1)

[('cucumber', 19)]

Counter can subtract others, such as mappings or iterables. It is not an error if some elemtns are missing.

In [13]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -6})

In [14]:
# Missing subtract element
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 4, 'b': 0, 'c': -3, 'd': -6})

In [15]:
# Missing original element
c = Counter(a=4, b=2, c=0)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -4})

In [16]:
# Subtracting dictionary
c = Counter(a=4, b=2, c=0, d=-2)
d = {'a':3}
c.subtract(d)
c

Counter({'a': 1, 'b': 2, 'c': 0, 'd': -2})

Counter can also add, but it is called update. Instead of replacing, it simply adds counts.

In [17]:
c.update('abbccc')
c

Counter({'a': 2, 'b': 4, 'c': 3, 'd': -2})

Mathematical operations can be used with Counter objects. Negative counts are removed.

In [21]:
c = Counter(a=2, b=-4, c=0, d=10)
d = Counter(a=-1, b=2, c=3, d=17)

In [22]:
# Add
c + d

Counter({'a': 1, 'c': 3, 'd': 27})

In [23]:
# Subtract
c - d

Counter({'a': 3})

In [24]:
# Intersection
c & d

Counter({'d': 10})

In [25]:
# Union
c | d

Counter({'a': 2, 'b': 2, 'c': 3, 'd': 17})

Unary operator + gets rid of negative and zero elements, and - gets rid of positive and zero elements.

In [26]:
c = Counter(a=2, b=-4, c=0)
+c

Counter({'a': 2})

In [27]:
-c

Counter({'b': 4})

#### Defaultdict

Instead of using dictionary's <code>setdefault</code>, </code>defaultdict</code> can achieve higher efficiency and more flexibility. Here, it uses a list for the default value. 

In [28]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
d = defaultdict(list)
d

defaultdict(list, {})

In [29]:
for k, v in l:
    d[k].append(v)

sorted(d.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

Same thing could have achieved using plain dictionary's <code>setdefault</code>.

In [18]:
d2 = {}
for k, v in l:
    d2.setdefault(k, []).append(v)
sorted(d2.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

<code>defaultdict</code> can also use other type for the default value, such as integer. If integer is used, it can be used as a counter.

In [41]:
l = ['bacon', 'egg', 'egg', 'oj', 'bacon', 'bacon', 'bacon', 'bacon']
d = defaultdict(int)
for k in l:
    d[k] += 1
sorted(d.items())

[('bacon', 5), ('egg', 2), ('oj', 1)]

If I need a counter, it is better to use Counter().

In [33]:
d2 = Counter(l)
d2

Counter({'bacon': 5, 'egg': 2, 'oj': 1})

In [35]:
list(d2.elements())

['bacon', 'bacon', 'bacon', 'bacon', 'bacon', 'egg', 'egg', 'oj']

In [36]:
d2.most_common(2)

[('bacon', 5), ('egg', 2)]

Set can also be used as a default type.

In [43]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 3)]
d = defaultdict(set)
for k, v in l:
    d[k].add(v)
sorted(d.items())

[('bacon', {3}), ('egg', {10}), ('oj', {1, 3})]

Default value does not have to be set by the type. By passing a value with lambda, any constant value can be set to default.

In [46]:
def constant_factory(value):
    return lambda: value
d = defaultdict(constant_factory('<missing>'))
d.update(name='John', action='ran')
'%(name)s %(action)s to %(object)s' % d

'John ran to <missing>'

In [48]:
d2 = defaultdict(constant_factory(['Must Have!']))
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
for k,v in l:
    d2[k].append(v)
sorted(d2.items())

[('bacon', ['Must Have!', 3, 10, 1, 3, 11]),
 ('egg', ['Must Have!', 3, 10, 1, 3, 11]),
 ('oj', ['Must Have!', 3, 10, 1, 3, 11])]

#### Namedtuple

Making breakfast is tough. Especially when there are many orders. Namedtuple can help with getting the menus straight. Here's a namedtuple regarding what James wants.

In [3]:
Breakfast = namedtuple('Breakfast', 'eggs bacons pancakes oj')
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

In [31]:
James.eggs, James.bacons

(3, 2)

Breakfast menu can also be a string with commas in it.

In [32]:
Breakfast = namedtuple('Breakfast', 'eggs, bacons, pancakes, oj')
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

Or a iterable such as a list.

In [33]:
Breakfast = namedtuple('Breakfast', ['eggs', 'bacons', 'pancakes', 'oj'])
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

With <code>_make</code>, it is possible to use an iterable to create a namedtuple.

In [34]:
quantities = [1, 2, 3, 1]
Mary = Breakfast._make(quantities)
Mary

Breakfast(eggs=1, bacons=2, pancakes=3, oj=1)

With <code>_asdict</code>, namedtuple returns a dictionary.

In [35]:
Mary._asdict()

{'eggs': 1, 'bacons': 2, 'pancakes': 3, 'oj': 1}

With <code>_replace</code>, namedtuple returns a new namedtuple with replaced elements. Mary changed her mind and now wants more eggs.

In [36]:
Mary._replace(eggs=99)

Breakfast(eggs=99, bacons=2, pancakes=3, oj=1)

With <code>_fields</code>, namedtuple can be combined into a new namedtuple. By combining Breakfast and Taste, I made VIP_Breakfast, which has more options, such as temperature, spiciness, and more. 

In [37]:
Breakfast._fields

('eggs', 'bacons', 'pancakes', 'oj')

In [38]:
Taste = namedtuple('Taste', 'temperature, spiciness, sweetness, savoriness')
VIP_Breakfast = namedtuple('VIP_Breakfast', Breakfast._fields + Taste._fields)
Nom = VIP_Breakfast(1, 3, 5, 7, 'hot', 'spicy', 'not sweet', 'very savory')
Nom

VIP_Breakfast(eggs=1, bacons=3, pancakes=5, oj=7, temperature='hot', spiciness='spicy', sweetness='not sweet', savoriness='very savory')

Passing defaults when initiating namedtuple sets defaults values starting from the farthest right parameters. By providing defaults with an array length of four, default values are set for Taste. It is possible to override default values by providing right positional argument or keyword argument.

In [39]:
Taste = namedtuple('Taste', 'temperature, spiciness, sweetness, savoriness')
VIP_Breakfast = namedtuple('VIP_Breakfast', 
                           Breakfast._fields + Taste._fields,
                           defaults=['hot', 'regular', 'semi-sweet', 'savory'])
Nom = VIP_Breakfast(1, 3, 5, 7, 'cold', savoriness='not savory')
Nom

VIP_Breakfast(eggs=1, bacons=3, pancakes=5, oj=7, temperature='cold', spiciness='regular', sweetness='semi-sweet', savoriness='not savory')

Using <code>_field_defaults</code>, it is easy to check the default values.

In [40]:
Nom._field_defaults

{'temperature': 'hot',
 'spiciness': 'regular',
 'sweetness': 'semi-sweet',
 'savoriness': 'savory'}

Using getattr is another way to get the field of namedtuple.

In [41]:
getattr(Nom, 'oj')

7

If a server wrote the order with a dictionary format, it is no problem. Namedtuple can be initialized with double stars.

In [42]:
VIP_order = {'eggs': 2, 'bacons': 3, 'pancakes': 9, 'oj': 1}
Nom = VIP_Breakfast(**VIP_order)
Nom

VIP_Breakfast(eggs=2, bacons=3, pancakes=9, oj=1, temperature='hot', spiciness='regular', sweetness='semi-sweet', savoriness='savory')

#### Deque

Pronounced as "deck," deque is used to work on only certain number of elements. For example, it is used for saving last five visits in history or saving last couple lines of a file. Another usage is inserting and popping elements in the beginning of the deque. 

In [27]:
d = deque('abc', 3)
d

deque(['a', 'b', 'c'])

In [28]:
d.append('d')
d

deque(['b', 'c', 'd'])

In [29]:
d.appendleft('1')
d

deque(['1', 'b', 'c'])

In [30]:
d.extend(['d', 'e'])
d

deque(['c', 'd', 'e'])

In [31]:
# Extendleft's arguments are inserted in a reversed order.
d.extendleft([0, 1])
d

deque([1, 0, 'c'])

In [32]:
d.index(0)

1

In [33]:
d.pop()

'c'

In [34]:
d.popleft()

1

In [35]:
d

deque([0])

In [36]:
d.remove(0)
d

deque([])

In [38]:
d.extend([0,1,2])
d

deque([0, 1, 2])

In [41]:
d.reverse()
d

deque([2, 1, 0])

In [43]:
# cannot insert when it is full
d.insert(1, 7)
d

IndexError: deque already at its maximum size

In [47]:
d.rotate()
d

deque([0, 2, 1])

In [49]:
d.rotate(-1)
d

deque([1, 0, 2])

In [50]:
d.maxlen

3

In [51]:
d.clear()
d

deque([])

It is very similar to list, except that it has a fixed size, and it can accept elements in the beginning efficiently.

In [52]:
# Just more exmaples.
# last 3 numbers of data can be achieved like this
for i in range(10):
    d.append(i)
d

deque([7, 8, 9])

Deque can also be used for grabbing last 5 lines of file or getting moving averages. 

### Itertools

#### Permutations

Permutations function simply return permutations. It can be used for cases where order matters, and there is no overlap. For example, making a sequence of music notes with given possible notes without overlap is an example of using permutations.

In [249]:
list(permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

Here is my implementation that returns a list. However, itertools version returns a generator. 

In [240]:
def our_permutations(iterable, num=None):
    """Return a list of permutations."""
    length = len(iterable)
    num = length if num is None else num
    if num > length:
        return []
    elif num == 0:
        return [()]
    else:
        result = []
        for i in range(length):
            rest = tuple(our_permutations(iterable[0:i] + iterable[i+1:], num - 1))
            for j in range(len(rest)):
                result.append(tuple(iterable[i]) + rest[j])
        return result

In [243]:
our_permutations('BC', 1)

[('B',), ('C',)]

In [244]:
our_permutations('ABCD')

[('A', 'B', 'C', 'D'),
 ('A', 'B', 'D', 'C'),
 ('A', 'C', 'B', 'D'),
 ('A', 'C', 'D', 'B'),
 ('A', 'D', 'B', 'C'),
 ('A', 'D', 'C', 'B'),
 ('B', 'A', 'C', 'D'),
 ('B', 'A', 'D', 'C'),
 ('B', 'C', 'A', 'D'),
 ('B', 'C', 'D', 'A'),
 ('B', 'D', 'A', 'C'),
 ('B', 'D', 'C', 'A'),
 ('C', 'A', 'B', 'D'),
 ('C', 'A', 'D', 'B'),
 ('C', 'B', 'A', 'D'),
 ('C', 'B', 'D', 'A'),
 ('C', 'D', 'A', 'B'),
 ('C', 'D', 'B', 'A'),
 ('D', 'A', 'B', 'C'),
 ('D', 'A', 'C', 'B'),
 ('D', 'B', 'A', 'C'),
 ('D', 'B', 'C', 'A'),
 ('D', 'C', 'A', 'B'),
 ('D', 'C', 'B', 'A')]

In [205]:
our_permutations('ABCD', 2)

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C')]

And this is from python documentation. 

In [171]:
def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable) # ('A', 'B', 'C', 'D')
    n = len(pool) # 4
    r = n if r is None else r # 2
    if r > n:
        return
    indices = list(range(n)) # [0, 1, 2, 3]
    cycles = list(range(n, n-r, -1)) # [1, 0]
    yield tuple(pool[i] for i in indices[:r]) # tuple(pool[i] for i in [0, 1]) -> ('A', 'B')
    while n: # 4
        for i in reversed(range(r)): # i in reversed(range(2)) -> 1, 0
            cycles[i] -= 1 # -1, 0
            if cycles[i] == 0: 
                indices[i:] = indices[i+1:] + indices[i:i+1] 
                # indices[1:] = indices[2:] + indices[1:2]
                # [1,2,3] = [2,3] + [1]
                cycles[i] = n - i # cycles[0] = 4
            else: # i: 1
                j = cycles[i] # j: -1
                indices[i], indices[-j] = indices[-j], indices[i] # 1, 1 = 1, 1
                yield tuple(pool[i] for i in indices[:r]) 
                # tuple(pool[i] for i in [0, 1]) -> ('A', 'B')
                break
        else:
            return

In [172]:
permutations('ABC', 5)

<generator object permutations at 0x7f31ab408350>

#### Combinations

Similar to permutations, it returns combinations. 

In [44]:
list(combinations('ABCD', 3))

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]

Here is combinations that I wrote similar to our_permutations. 

In [40]:
def our_combinations(iterable, num=None):
    """Return a list of combinations."""
    length = len(iterable)
    num = length if num is None else num
    if num > length:
        return []
    elif num == 0:
        return [()]
    else:
        result = []
        for i in range(length - num + 1):
            rest = tuple(our_combinations(iterable[i+1:], num - 1))
                result.append(tuple(iterable[i]) + rest[j])
        return result

In [45]:
our_combinations('ABCD', 3)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]

In python <a href="https://docs.python.org/3/library/itertools.html#itertools.combinations">documentation</a>, combinations is written in terms of permutations. It uses permutations to create all the possible indices of making permutations. Then it checks whether the set of indices matches the sorted indices. Although it is an interesting way to implement combinations, it is not very efficient. 

In [ ]:
def p_combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in permutations(range(n), r):
        if sorted(indices) == list(indices):
            yield tuple(pool[i] for i in indices)

#### Chain

Chain is used to put multiple iterables together. 

In [3]:
l1 = ['a', 'b', 'c']
l2 = ['d', 'e', 'f']
for element in chain(l1, l2):
    print(element)

a
b
c
d
e
f


I wonder whether using chain is faster than just using + with lists.

In [4]:
long_lst = list(range(10000))
long_lst2 = list(range(10000, 20000))

In [5]:
%timeit [x for x in long_lst+long_lst2]

830 µs ± 4.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
%timeit [x for x in chain(long_lst, long_lst2)]

792 µs ± 7.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


So, using chain is slighly faster than simply using +. How about using more + vs. chain?

In [7]:
%timeit [x for x in long_lst+long_lst2+long_lst+long_lst2]

2.16 ms ± 45 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit [x for x in chain(long_lst,long_lst2,long_lst,long_lst2)]

1.59 ms ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Using more chain is more efficient than +.

#### Count

Even though it is count, it is imported as count_from here. It simply counts from a given input. 

In [13]:
list(zip(['a', 'b', 'c', 'd'], count_from(1)))

[('a', 1), ('b', 2), ('c', 3), ('d', 4)]

In [2]:
list(zip(['a', 'c', 'e', 'g'], count_from(1, 2)))

[('a', 1), ('c', 3), ('e', 5), ('g', 7)]

Different from <code>range</code>, count_from does not take the end limit. 

#### Product

It is basically nested for loops. 

In [50]:
list(cross_product('ABCD', repeat=1))

[('A',), ('B',), ('C',), ('D',)]

In [49]:
list(cross_product('ABCD', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C'),
 ('D', 'D')]

In [51]:
list(cross_product('ABCD', repeat=3))

[('A', 'A', 'A'),
 ('A', 'A', 'B'),
 ('A', 'A', 'C'),
 ('A', 'A', 'D'),
 ('A', 'B', 'A'),
 ('A', 'B', 'B'),
 ('A', 'B', 'C'),
 ('A', 'B', 'D'),
 ('A', 'C', 'A'),
 ('A', 'C', 'B'),
 ('A', 'C', 'C'),
 ('A', 'C', 'D'),
 ('A', 'D', 'A'),
 ('A', 'D', 'B'),
 ('A', 'D', 'C'),
 ('A', 'D', 'D'),
 ('B', 'A', 'A'),
 ('B', 'A', 'B'),
 ('B', 'A', 'C'),
 ('B', 'A', 'D'),
 ('B', 'B', 'A'),
 ('B', 'B', 'B'),
 ('B', 'B', 'C'),
 ('B', 'B', 'D'),
 ('B', 'C', 'A'),
 ('B', 'C', 'B'),
 ('B', 'C', 'C'),
 ('B', 'C', 'D'),
 ('B', 'D', 'A'),
 ('B', 'D', 'B'),
 ('B', 'D', 'C'),
 ('B', 'D', 'D'),
 ('C', 'A', 'A'),
 ('C', 'A', 'B'),
 ('C', 'A', 'C'),
 ('C', 'A', 'D'),
 ('C', 'B', 'A'),
 ('C', 'B', 'B'),
 ('C', 'B', 'C'),
 ('C', 'B', 'D'),
 ('C', 'C', 'A'),
 ('C', 'C', 'B'),
 ('C', 'C', 'C'),
 ('C', 'C', 'D'),
 ('C', 'D', 'A'),
 ('C', 'D', 'B'),
 ('C', 'D', 'C'),
 ('C', 'D', 'D'),
 ('D', 'A', 'A'),
 ('D', 'A', 'B'),
 ('D', 'A', 'C'),
 ('D', 'A', 'D'),
 ('D', 'B', 'A'),
 ('D', 'B', 'B'),
 ('D', 'B', 'C'),
 ('D', 'B'

In [52]:
# A demostration version from python documentation
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)

### Typing

Skip typing for now.

### Statistics

#### Mean

It is also called average. 

In [57]:
# Average of numbers from 0 to 10
mean(range(11))

5

In [58]:
mean(range(101))

50

In [61]:
mean([0, 10, 10, 10, 10])

8

#### Median

Median is a center value of data. If there are even number of data, it is average of two middle ones.

In [59]:
median(range(11))

5

In [60]:
median(range(101))

50

Median is different here.

In [63]:
median([0, 10, 10, 10, 10])

10

### Math

#### ceil

It returns a ceiling of a float, and itself if it is an integer. It works with negative numbers as well.

In [64]:
ceil(0.3)

1

In [69]:
ceil(99.9)

100

In [65]:
ceil(-0.3)

0

In [68]:
ceil(-10.9)

-10

In [66]:
ceil(1)

1

#### inf

inf is a constant of infinity. 

In [70]:
inf

inf

In [80]:
# No matter how we try to make it bigger, it is still infinity
inf + 10000000, inf ** inf

(inf, inf)

In [72]:
# Negative version of infinity
-inf

-inf

In [73]:
# Still negative
-inf + 10000000

-inf

In [74]:
# Making negative infinity positive
-inf * -inf

inf

In [83]:
# Just too big to be finite, but sign changes
-inf * -0.000000001

inf

In [82]:
-inf * -1000

inf

In [78]:
# This is not a number
inf * 0

nan

### Functools

#### Cache

Before learning lru_cache, I want to learn about cache, first.

In [2]:
# functools has cache if python's version >= 3.9
# from functools import cache
cache = lru_cache(None)

In [3]:
@cache
def factorial(n):
    return n * factorial(n-1) if n else 1

In [ ]:
%time factorial(425)

In [5]:
%time factorial(425)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.4 µs


15977543825580449039059028491802903127380338791863483049128370991067108396625766050267688895710574429084998108101039700641517381873763319601028149484002114693798393699491396328947757831366620306555297082031175675213705798463163227066295062015432599288847760596340315258786855260497719902961743066792834934241285332466334330976532707258804959387324668690208796476945391790915591031353635928193404280314886136458335477631166936908618143948819872191817526972357414781307615320059024031912964120178430314558792157725588006914593404752298568543044309034599056012503965564056636054142381603688108699830585169089536136973697948594333529771072449681268640636424890253957698622305602915377337745834781001890868690302403007357837222872688402785347907470873020865015995233051392945984572795828503249504365669280534212772911921618337423949824000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Because it is cached, it is very fast the second time. It can be written like this:

In [6]:
def memoize (fn):
    "Memoize a function"
    results = dict()
    def mem (x):
        if x in results.keys():
            return results[x]
        else:
            results[x] = fn(x)
            return results[x]
    return mem

In [7]:
@memoize
def factorial2(n):
    return n * factorial2(n-1) if n else 1

In [8]:
%time factorial2(425)

CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 924 µs


15977543825580449039059028491802903127380338791863483049128370991067108396625766050267688895710574429084998108101039700641517381873763319601028149484002114693798393699491396328947757831366620306555297082031175675213705798463163227066295062015432599288847760596340315258786855260497719902961743066792834934241285332466334330976532707258804959387324668690208796476945391790915591031353635928193404280314886136458335477631166936908618143948819872191817526972357414781307615320059024031912964120178430314558792157725588006914593404752298568543044309034599056012503965564056636054142381603688108699830585169089536136973697948594333529771072449681268640636424890253957698622305602915377337745834781001890868690302403007357837222872688402785347907470873020865015995233051392945984572795828503249504365669280534212772911921618337423949824000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [9]:
%time factorial2(425)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.5 µs


15977543825580449039059028491802903127380338791863483049128370991067108396625766050267688895710574429084998108101039700641517381873763319601028149484002114693798393699491396328947757831366620306555297082031175675213705798463163227066295062015432599288847760596340315258786855260497719902961743066792834934241285332466334330976532707258804959387324668690208796476945391790915591031353635928193404280314886136458335477631166936908618143948819872191817526972357414781307615320059024031912964120178430314558792157725588006914593404752298568543044309034599056012503965564056636054142381603688108699830585169089536136973697948594333529771072449681268640636424890253957698622305602915377337745834781001890868690302403007357837222872688402785347907470873020865015995233051392945984572795828503249504365669280534212772911921618337423949824000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

#### lru_cache

It is same as cache except its size can be limited. 

## Playing with utilities

After getting familiar with libraries, I'd like to look at the utilites or helper functions. 

### Jumping into action

<code>answer</code> is just an assert that compares expected value with actual value.

In [ ]:
def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

In [21]:
answer(49, 3, 1 + 1 - 1)

AssertionError: For 49, expected 1 but got 3.

In [11]:
answer(81, 10, 5 + 5)

True

<code>mapt</code> is similar to map(fn, *args), but returns a tuple. So, it does not have to be wrapped around with list, tuple, or others in order to look at the contents. Another way to use is to convert types of its contents.

In [15]:
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

In [16]:
mapt(lambda x: x + ['🚀'], [[1, 2], [3, 4]])

([1, 2, '🚀'], [3, 4, '🚀'])

In [17]:
mapt(lambda x: x + 1, [1,2,3])

(2, 3, 4)

In [46]:
mapt(int, ['1', '2', '5'])

(1, 2, 5)

<code>trunc</code> is very fun as well. If any string is longer than 100 characters, it truncates, and inserts dots in the middle. But it can also be customized. I can set different lengths for left and right, and different characters for dots. I can even use emojis for dots!

In [19]:
def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
#     dots = ' ... '
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

In [20]:
long_string = '''
I am not sure what this is about, but I am a long string, okay? If you have any problem regarding me, talk to my boss. I simply do not get paid enough for any of this nonsense!'''

In [21]:
trunc(long_string.strip(), left=32, right=11, dots=' 🤐💀🤯 ')

'I am not sure what this is about 🤐💀🤯 s nonsense!'

<code>parse</code> is a flexible way to parse inputs. Instead of changing the type of input manually or setting a different seperator, we can do all these at once. Another cool feature is its printing ability. It prints information about the input and parsed input. 

In [22]:
def parse(day, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, 
    and apply `parser` to each."""
    # updated filename for my for input file system
    fname = f'input{day}.txt'
    text  = open(fname).read()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{fname} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        # I added parser.__name__ for fun
        print(f'{dash}\nparse({day}) ➜ {len(entries)} entries as {parser.__name__}:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

In [23]:
print(int.__name__)

int


In [24]:
input1 = parse(1, parser=int)

----------------------------------------------------------------------------------------------------
input1.txt ➜ 9777 chars, 2000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
151
152
153
158
159
163
164
----------------------------------------------------------------------------------------------------
parse(1) ➜ 2000 entries as int:
----------------------------------------------------------------------------------------------------
(151, 152, 153, 158, 159, 163, 164, 162, 161, 167, 169, 168, 169, 170, ... , 8078, 8081, 8112, 8127)
----------------------------------------------------------------------------------------------------


Here are some "type declaration." Although python does not actually have type declarations, we can use typing library and make a new type, such as Atom. We have to be careful about using Char, as it is essentially str. 

In [27]:
Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

Next, we have functions that extracts certain types, such as ints, digits, atoms. Code itself is pretty self explanatory, and documentation explains exactly what it does.

In [31]:
def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

In [32]:
ints('a0b1c2-34z?-')

(0, 1, 2, -34)

In [33]:
def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

In [34]:
digits('a0b1c2-34z?-')

(0, 1, 2, 3, 4)

In [35]:
def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

In [36]:
words('I have 39 apples in my mouth.')

['I', 'have', 'apples', 'in', 'my', 'mouth']

In [39]:
def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text

In [40]:
atom('38.4')

38.4

In [41]:
atom('90')

90

In [42]:
atom('atom')

'atom'

In [43]:
atom('We are building lisp with atoms. 🔬 How far can we go? 🚀')

'We are building lisp with atoms. 🔬 How far can we go? 🚀'

In [44]:
def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

In [45]:
atoms('12.8 billion atoms swimming in the ocean. Yeah? 🤯!')

(12.8, 'billion', 'atoms', 'swimming', 'in', 'the', 'ocean.', 'Yeah')

In [1]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

In [2]:
quantify(range(10), lambda x: x & 2 == 0)

6

In [10]:
quantify([0, 1, 'two', '3', 4, 'five'], lambda x: isinstance(x, int))

3

<code>multimap</code> returns a defaultdict with list as a default value. It makes using defaultdict with list easier. As we used defaultdict earlier, it is a little bit awkward to populate values ourselves using for loop with an iterable. It also gives us option to make values symmetric, which is useful.

In [47]:
class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

In [57]:
multimapped = multimap([(1, 'one'), (2, 'two'), (3, 'three')])
multimapped

multimap(list, {1: ['one'], 2: ['two'], 3: ['three']})

In [59]:
multimapped[1].append('first')
multimapped

multimap(list, {1: ['one', 'first'], 2: ['two'], 3: ['three']})

In [60]:
multimapped = multimap([(1, 'one'), (2, 'two'), (3, 'three')], symmetric=True)
multimapped

multimap(list,
         {1: ['one'],
          'one': [1],
          2: ['two'],
          'two': [2],
          3: ['three'],
          'three': [3]})

Python provides sum built in, but not prod. We can call math.prod as we have that as an option, or define our own. Compared to our prod, math.prod is faster.

In [61]:
def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

In [66]:
%timeit prod(range(100))

6.41 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [67]:
import math

In [68]:
%timeit math.prod(range(100))

2.28 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


<code>total</code> accepts a counter and returns a total number of counts. It would have been nice if it were built into Counter.

In [70]:
def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

In [77]:
batman = 'I am batman.'
c = Counter(batman)
total(c)

12

In [78]:
len(batman)

12

<code>sign</code> returns 1, 0, or -1 depending on the argument's sign.

In [79]:
def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

In [80]:
sign(3)

1

In [81]:
sign(-1)

-1

In [82]:
sign(0)

0

<code>transpose</code> transposes a matrix. Why would we transpose a matrix? To solve problems. What kind of problems? Well, matrix problems.

In [83]:
def transpose(matrix) -> list: return list(zip(*matrix))

In [84]:
matrix = [[0, 1],
          [2, 3]]
transpose(matrix)

[(0, 2), (1, 3)]

Nothing returns None.

In [85]:
def nothing(*args) -> None: return None

In [87]:
print(nothing('Something'))

None


Here are some aliases. We could use either version, and the code will work. Instead of using lru_cache(None), we can just use cache from functools. It is also faster than lru_cache.

In [88]:
cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

In [98]:
cat(['a', 'b', 'c'])

'abc'

In [99]:
list(chain.from_iterable([[3], [5], [7, 9]]))

[3, 5, 7, 9]

In [107]:
# Does not flatten deeply
list(chain.from_iterable([[3], [5], [[[7], 9], 11]]))

[3, 5, [[7], 9], 11]

In [103]:
import time
@cache
def waiting(seconds: int):
    """Wait for seconds. If already waited for that long,
    skip waiting."""
    time.sleep(seconds)
    return f'I waited for {seconds} seconds'

In [104]:
waiting(1)

'I waited for 1 seconds'

In [106]:
# Program starts to lie after calling once
waiting(1)

'I waited for 1 seconds'

Lastly, we go over utilities used for 2D. Basically, anything with x, y coordinates, we can use thes. 

In [108]:
Point = Tuple[int, int] # (x, y) points on a grid

neighbors4 = ((0, 1), (1, 0), (0, -1), (-1, 0))               
neighbors8 = ((1, 1), (1, -1), (-1, 1), (-1, -1)) + neighbors4

class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping=(), rows=(), neighbors=neighbors4):
        """Initialize with, e.g., either `mapping={(0, 0): 1, (1, 0): 2, ...}`,
        or `rows=[(1, 2, 3), (4, 5, 6)].
        `neighbors` is a collection of (dx, dy) deltas to neighboring points.`"""
        self.update(mapping if mapping else
                    {(x, y): val 
                     for y, row in enumerate(rows) 
                     for x, val in enumerate(row)})
        self.width  = max(x for x, y in self) + 1
        self.height = max(y for x, y in self) + 1
        self.deltas = neighbors
        
    def copy(self) -> Grid: return Grid(self, neighbors=self.deltas)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        x, y = point
        return [(x+dx, y+dy) for (dx, dy) in self.deltas 
                if (x+dx, y+dy) in self]
    
    def to_rows(self) -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self[x, y] for x in range(self.width)]
                for y in range(self.height)]

In [109]:
neighbors4

((0, 1), (1, 0), (0, -1), (-1, 0))

In [111]:
Point

typing.Tuple[int, int]

In [137]:
# Initiating
grid = Grid(mapping={(0,0): 1})
grid

{(0, 0): 1}

In [133]:
# More ways to initiate
grid = Grid(rows=[(1,2,3), (4,5,6)])
grid

{(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 4, (1, 1): 5, (2, 1): 6}

In [134]:
grid.neighbors((1,0))

[(1, 1), (2, 0), (0, 0)]

In [135]:
grid.height, grid.width, grid.deltas

(2, 3, ((0, 1), (1, 0), (0, -1), (-1, 0)))

In [136]:
# Back to rows
rows = grid.to_rows()
rows

[[1, 2, 3], [4, 5, 6]]

That's it. We went over utilities so that we can focus more on solving problems rather than building tools. With those tools on hand, we only have to figure out which ones to use. If there is nothing, then we can move on to building tools. Now that we are ready, we can move onto solving Advent of Code problems.